In [1]:
import glob
import pandas as pd
import time
import docx
import PyPDF2
import cv2
import pytesseract
from datetime import date
import re
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [2]:
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r"C:\Users\QuadAdmin\Downloads\files\**\*.pdf",recursive = True)
print(f"time took: {(time.time() - st_time):.2f} seconds")

time took: 0.00 seconds


In [3]:
paths

['C:\\Users\\QuadAdmin\\Downloads\\files\\pdf\\call1.pdf',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\pdf\\call3.pdf',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\pdf\\f013.pdf',
 'C:\\Users\\QuadAdmin\\Downloads\\files\\pdf\\try2.pdf']

In [4]:
DF["fileName"] = paths

In [5]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\pdf\call1.pdf,NaN
1,C:\Users\QuadAdmin\Downloads\files\pdf\call3.pdf,NaN
2,C:\Users\QuadAdmin\Downloads\files\pdf\f013.pdf,NaN
3,C:\Users\QuadAdmin\Downloads\files\pdf\try2.pdf,NaN


In [6]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [8]:
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blaher")
spark = SparkSession.builder.config(conf = conf).master("local[1]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

In [9]:
spark

In [10]:
sc = spark.sparkContext

In [11]:
# ls_ran = list(range(1,41))
# len(ls_ran)

In [12]:
# ls_rdd = sc.parallelize(ls_ran)
# ls_rdd.take(5)

In [13]:
# type(ls_rdd)

In [14]:
# ls_rdd.getNumPartitions()

In [15]:
def getText_PDF(filename_with_path):
    pdfFileObj = open(filename_with_path, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0)
    text = (pageObj.extractText()).replace("\n","")
    return text

In [16]:
DF["fileName"][0]

'C:\\Users\\QuadAdmin\\Downloads\\files\\pdf\\call1.pdf'

In [17]:
# readfiles(DF["fileName"][0])
# re.sub("\n"," ",getText_Docx(DF["fileName"][3]))
getText_PDF(DF["fileName"][3])

"Hello, my name is David Johnson and I live in Maine.  My credit card number is 4095-2609-9393-4932 and my Crypto wallet id is 16Yeky6GMjeNkAiNcBY7ZhrLoMSgg1BoyZ.    On September 18 I visited microsoft.com and sent an email to test@microsoft.com,  from the IP 192.168.0.1.   My passport: 191280345 and my phone number: (212) 555-1234.   This is a valid IBAN: IL150120690000003111111 . Can you please check the status on bank account 154567876544 in PresidiBank?   Kate's social security number is 078-05-1120.  Her driver license? it is 1234567A. Code of Conduct.   For information see the Code of Conduct FAQ or contact opencode@microsoft.com with any additional questions or comments. Agent: It's a good day at Bank of Wealth, my name is Heather, how can I help you? Customer: I would like to know my remaining money in my account.  Agent: I'd be glad to help you May I please get your Bank Account number and the Name on the Account?  Customer: Sure, its Tracy Q. Randall, account number 805-7845-

In [18]:
paths_rdd = sc.parallelize(paths)

In [19]:
# text_rdd = paths_rdd.map(readfiles)

In [20]:
start_time = time.time()
DF["text"] = DF["fileName"].apply(lambda path: getText_PDF(path))
print(f"time took: {(time.time() - start_time):.2f} seconds")

time took: 0.13 seconds


In [21]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\pdf\call1.pdf,Agent: It's a good day today at Bank of Wealth...
1,C:\Users\QuadAdmin\Downloads\files\pdf\call3.pdf,"Agent: This is Mike of Pizza Loco, what & your..."
2,C:\Users\QuadAdmin\Downloads\files\pdf\f013.pdf,My name is Joseph Morgan ssn is 123 48 6789 an...
3,C:\Users\QuadAdmin\Downloads\files\pdf\try2.pdf,"Hello, my name is David Johnson and I live in ..."


In [22]:
DF["text"] = DF["text"].apply(lambda text: re.sub("\n"," ",text))

In [23]:
DF

,fileName,text
0,C:\Users\QuadAdmin\Downloads\files\pdf\call1.pdf,Agent: It's a good day today at Bank of Wealth...
1,C:\Users\QuadAdmin\Downloads\files\pdf\call3.pdf,"Agent: This is Mike of Pizza Loco, what & your..."
2,C:\Users\QuadAdmin\Downloads\files\pdf\f013.pdf,My name is Joseph Morgan ssn is 123 48 6789 an...
3,C:\Users\QuadAdmin\Downloads\files\pdf\try2.pdf,"Hello, my name is David Johnson and I live in ..."


In [24]:
from pyspark.sql.types import *

In [25]:
from pyspark.sql import *

In [26]:
import pyspark.sql.functions as F

In [27]:
from parse_pii.piiparser import *

In [28]:
DF["text"][0]

"Agent: It's a good day today at Bank of Wealth, my name is Heather, how can I help you? Customer: I would like to know my remaining money in my account.  Agent: I'd be glad to help you May I please get your Bank Account number and the Name on the Account?  Customer: Sure, its Tracy Q. Randall, account number 805-7845-3895-061 Agent: Thank you, let me just check on it. Ok, can you please, verify the last four numbers of your social security ID?  Customer: It is ****.  Agent: You still have 84 thousand and 65 cents. Is there anything else that I could assist you with?  Customer: Yes, If I transfer it to my bank account in Lloyds of London, how long will it take?  Agent: If we do the transaction over the phone or online, our team will still contact you for verification prior sending your money to a different bank. The whole process usually just takes 2-3days.  Customer: Oh, I see, never mind, I'll just do it after the holidays. Thanks for your help. Heather!  Agent: You are very welcome,

In [29]:
classify(DF["text"][0])

'yes'

In [30]:
sparkDF = spark.createDataFrame(DF)

In [31]:
sparkDF.show(5)

+--------------------+--------------------+
|            fileName|                text|
+--------------------+--------------------+
|C:\Users\QuadAdmi...|Agent: It's a goo...|
|C:\Users\QuadAdmi...|Agent: This is Mi...|
|C:\Users\QuadAdmi...|My name is Joseph...|
|C:\Users\QuadAdmi...|Hello, my name is...|
+--------------------+--------------------+



In [32]:
sparkDF.rdd.getNumPartitions()

1

In [33]:
classify_udf = F.udf(classify, StringType())

In [34]:
# start_time = time.time()
# sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [35]:
# fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [36]:
# start_time = time.time()
# fDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")

In [37]:
# fDF.printSchema()

In [38]:
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine
from presidio_analyzer.recognizer_registry import RecognizerRegistry

In [39]:
def entity_counter(entity_list):
    freq = {}
    entity_list = sorted(entity_list)
    for items in entity_list: 
        freq[items] = entity_list.count(items) 
 
    counter = ""
    for key, value in freq.items(): 
        if key == entity_list[-1]:
            counter += f"{key}:{value}"
        else:
            counter += f"{key}:{value}"+", "
    return counter

In [40]:
analyzer = AnalyzerEngine(nlp_engine=nlp)

In [41]:
def pii_classify(stext):
    text = stext
    response = analyzer.analyze(correlation_id=0, text = text,language="en",entities=[], score_threshold=0.5)
    if len(response) == 0:
        return "no"
    else:
        return "yes"

In [42]:
(DF["text"][1])

"Agent: This is Mike of Pizza Loco, what & your name and what would you like to order?! Customer: This is John Perez; I want One Family size of Pepperoni and 2 Regular Supreme. Agent: Thank you, so, that is One Pepperoni Family size, and Two Regular Supreme Pizza's. Is that right? Customer: Yep, deliver it at 745 Farmers Road, Modesto, California... Agent: Got it, would you like to a each. Customer: Sure, I'll just pay it in cash once it's delivered. Agent: Alight, it's gonna be right in front of your door, within 30mins. Thanks for calling, Pizza Loco! Have a great night! "

In [43]:
pii_classify(DF["text"][1])

'yes'

In [44]:
pii_apply_udf = F.udf(pii_classify, StringType())

In [45]:
sparkDF.select("fileName", classify_udf("text").alias("has_pii")).show(4)

+--------------------+-------+
|            fileName|has_pii|
+--------------------+-------+
|C:\Users\QuadAdmi...|    yes|
|C:\Users\QuadAdmi...|    yes|
|C:\Users\QuadAdmi...|    yes|
|C:\Users\QuadAdmi...|    yes|
+--------------------+-------+



In [46]:
fDF = sparkDF.select("fileName", classify_udf("text").alias("has_pii"))

In [47]:
fDF.write.mode("overwrite").saveAsTable("op_table4")

In [48]:
# fDF.repartition(1).write.mode("overwrite").csv("output",header=True)

In [49]:
# start_time = time.time()
# sparkDF.coalesce(2).write.mode("overwrite").format("csv").option("header","true").save("output")
# print(f"time took: {(time.time() - start_time):.2f} seconds")